# Experiment 05-01

In [1]:
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information

## Show the available Platforms

In [2]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:44)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [3]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [4]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [5]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;get_random_numbers;get_random_string;get_single_hash_nonce;mine_nonce


## Initial Tests

### Single Hash

In [6]:
# the plaintext to hash
#plaintext = 'bells hel the quick brown fox jumps over the lazy dog 12356'
plaintext = '665782'
#plaintext = 'this is a description of the latest block' + '395707976'
#plaintext = '120Make America Great Again'
#plaintext = '69817Make America Great Again'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)
leading_zeros = np.zeros(1, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

cl_leading_zeros = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    leading_zeros.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output,
    cl_leading_zeros)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)
cl.enqueue_copy(cl_queue, leading_zeros, cl_leading_zeros)

# print the results
print("Plaintext: %s" % plaintext)
print("Leading Zeros: %d" % leading_zeros[0])
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)
print("CL output      : %s" % np.frombuffer(hash_output, dtype=np.uint8))
print("CL output      : %s" % hash_output.tobytes())

assert hexlify(hash_output) == hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest())


Plaintext: 665782
Leading Zeros: 7
CL result      : b'0000000399c6aea5ad0c709a9bc331a3ed6494702bd1d129d8c817a0257a1462'
Correct result : b'0000000399c6aea5ad0c709a9bc331a3ed6494702bd1d129d8c817a0257a1462'
CL output      : [  50331648 2779694745 2591034541 2737947547 1888773357  701616427
 2685913304 1645509157]
CL output      : [  0   0   0   3 153 198 174 165 173  12 112 154 155 195  49 163 237 100
 148 112  43 209 209  41 216 200  23 160  37 122  20  98]
CL output      : b'\x00\x00\x00\x03\x99\xc6\xae\xa5\xad\x0cp\x9a\x9b\xc31\xa3\xedd\x94p+\xd1\xd1)\xd8\xc8\x17\xa0%z\x14b'


### Random String Generation

In [7]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
start = np.uint8(32)
end = np.uint8(126)
length = np.uint32(4)
result = np.zeros(length, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_start = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=start)
cl_end = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=end)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=length)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)

# execute the program
cl_program.get_random_numbers(
    cl_queue, (1,), None,
    cl_seed,
    cl_start,
    cl_end,
    cl_len,
    cl_result)

# get the results
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Seed: {seed}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Seed: 3984062511
Result: [102  57  97  52]
Result String: b'f9a4'


In [8]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
result = np.zeros(shape=16, dtype=np.uint8)
length = np.zeros(shape=(1,), dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, length.nbytes)

# execute the program
cl_program.get_random_string(
    cl_queue, (1,), None,
    cl_seed,
    cl_result,
    cl_len)

# get the results
cl.enqueue_copy(cl_queue, length, cl_len)
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Length: {length}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

Length: [1]
Result: [125   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Result String: b'}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


## Test get_single_hash_nonce

In [9]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
nonce = np.zeros(shape=16, dtype=np.uint8)
nonce_len = np.zeros(shape=1, dtype=np.uint8)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_hash_output = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_output.nbytes)

# execute the program
cl_program.get_single_hash_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

nonce_str = nonce[:nonce_len[0]].tobytes().decode('UTF-8')

# print the results
print("Seed: %s" % seed)
print("Plaintext: %s" % plaintext)
print("Nonce: %s" % nonce)
print("Nonce Length: %s" % nonce_len[0])
print("Nonce String: %s" % nonce_str)
print("Hash Output: %s" % hash_output)
print("Hash Output string: %s" % hexlify(hash_output))
#print("No Nonce          : %s" % hexlify(hashlib.sha256((plaintext).encode('utf-8')).digest()))
print("Correct result    : %s" % hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest()))

assert hexlify(hash_output) == hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest())


Seed: 4285531849
Plaintext: this is a description of the latest block
Nonce: [110  61  95  66 100  44  78  49  83   0   0   0   0   0   0   0]
Nonce Length: 9
Nonce String: n=_Bd,N1S
Hash Output: [ 350462665 1862737242 3443178250 3690290013 1737274487 2638484953
 1767871694 3392916186]
Hash Output string: b'c9a2e3145a1d076f0abb3acd5d5bf5db77b48c67d915449dce945f69daca3bca'
Correct result    : b'c9a2e3145a1d076f0abb3acd5d5bf5db77b48c67d915449dce945f69daca3bca'


## Mine Nonce

In [10]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(10000000)
nonce = np.zeros(shape=16 * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

# execute the program
cl_program.mine_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_window_size,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

# interpret the results
for i in range(64):
    nonce_str = nonce[i * 16:i * 16 + nonce_len[i]].tobytes().decode('UTF-8')
    hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))
    #hash = hashlib.sha256('665782'.encode('utf-8'))

    print("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

   0: [ 0]                                                                                  
   1: [ 1]                u 06b19cf0345165d8dedb5de1a06883d2030fe94ca75fab2bdb2af025235149c8
   2: [ 4]             fO"j 006b675116c9d9962fb15a3eba4f24e07db5c869582ae18aa0548d640e04adb6
   3: [ 4]             |?w: 00066ab5b34d689799818be343484fb0664fa9119f224c61210e7b1800e7dc56
   4: [ 6]           N`3]0B 0000a7673402b06bb264d372e857b18100c7e5548c2f1947025299ee39fef915
   5: [ 0]                                                                                  
   6: [ 0]                                                                                  
   7: [ 0]                                                                                  
   8: [ 0]                                                                                  
   9: [ 0]                                                                                  
  10: [ 0]                                                            